In [8]:
import sys
#This line imports the 'sys' module, which provides access to some variables used 
#or maintained by the interpreter and to functions that interact strongly with the interpreter. 
# It is not used in the rest of the code.
import pandas as pd
#This line imports the 'pandas' module and renames it as 'pd'. 
# Pandas is a popular data manipulation and analysis library for Python.
import spotipy
#Python library for the Spotify Web API. 
#It allows developers to interact with Spotify's music 
#streaming services and data about artists, albums, and songs.
from spotipy.oauth2 import SpotifyClientCredentials
#'SpotifyClientCredentials' class from the 'spotipy.oauth2' module. 
#This class is used to authenticate and authorize access to the Spotify Web API 
#using client credentials.
import os
#This line imports the 'os' module, which provides a way of interacting 
#with the operating system in a portable way. 
#It allows Python code to access environment variables, files, directories, 
#and other operating system resources.
from dotenv import load_dotenv
#This line imports the 'load_dotenv' function from the 'dotenv' module. 
#'dotenv' is a Python library that loads environment variables from a file named '.env' 
#in the current directory, if it exists. 
#It is often used to manage sensitive information such as API keys, passwords, and other secrets.
load_dotenv()
#This makes any environment variables defined in the '.env' file 
#available to the Python script as if they were defined in the operating system's environment. 
#This is useful for keeping sensitive information separate from the code and not hardcoding it
#into the script.

True

In [9]:
def get_playlist_info(link):
    # Initialize Spotify API client
    client_id = os.getenv("client_id")
    client_secret = os.getenv("client_secret")
    # The below two lines of code provide authentication credentials and create a connection to the Spotify API 
    # using the Spotipy library, allowing to retrieve data from Spotify's platform.
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    # Get playlist URI from link
    playlist_URI = link.split("/")[-1].split("?")[0]

    # Get all tracks from playlist 
    # up to a maximum of 300 tracks because I never add more than 300 and I have to request a max of 100 
    # as explained in the API
    offset = 0
    all_data = []
    for i in range(3):
        all_data.append(sp.playlist_tracks(playlist_URI, offset=offset)["items"])
        offset += 100

    # Store track information in a dictionary
    starting_table = {"song": [], 
                  "artist_1": [], 
                  "artist_id_1": [], 
                  "album": [], 
                  "album_id": [],
                  "duration": [],
                  "date": [],
                  "date_album": [], 
                  "imagen": [], 
                  "url_song": [], 
                  "total_tracks": []}

    for element in range(len(all_data)):
        for song in range(len(all_data[element])):
            starting_table["song"].append(all_data[element][song]["track"]["name"])
            starting_table["album"].append(all_data[element][song]["track"]["album"]["name"])
            starting_table["album_id"].append(all_data[element][song]["track"]["album"]['id'])
            starting_table["duration"].append(all_data[element][song]["track"]["duration_ms"])
            starting_table["date"].append(all_data[element][song]["added_at"])
            starting_table["date_album"].append(all_data[element][song]["track"]["album"]["release_date"])
            starting_table["imagen"].append(all_data[element][song]["track"]["album"]["images"][0]["url"])
            starting_table["url_song"].append(all_data[element][song]["track"]["external_urls"]["spotify"])
            starting_table["total_tracks"].append(all_data[element][song]["track"]["album"]["total_tracks"])
        
            num_artists = len(all_data[element][song]["track"]["artists"])
            for i in range(num_artists):
                artist_key = f"artist_{i+1}"
                artist_id_key = f"artist_id_{i+1}"
                starting_table.setdefault(artist_key, [])
                starting_table.setdefault(artist_id_key, [])
                starting_table[artist_key].append(all_data[element][song]["track"]["artists"][i]["name"])
                starting_table[artist_id_key].append(all_data[element][song]["track"]["artists"][i]["id"])
        
            # Add "none" to extra artist and artist_id columns if there is no value
            for j in range(num_artists+1, 6):
                artist_key = f"artist_{j}"
                artist_id_key = f"artist_id_{j}"
                starting_table.setdefault(artist_key, [])
                starting_table.setdefault(artist_id_key, [])
                starting_table[artist_key].append("none")
                starting_table[artist_id_key].append("none")

    # Create DataFrame from dictionary
    table = pd.DataFrame(starting_table)

    # Add label
    for index, row in table.iterrows():
    # Get the album ID from the album_id column
        album_id = row['album_id']
    
    # Retrieve the album information using the Spotify API
        album = sp.album(album_id)
    
    # Extract the label and popularity from the album information
        label = album['label']
        popularity = album['popularity']
    
    # Store the label and popularity in the new columns in the table
        table.at[index, 'label'] = label
        table.at[index, 'popularity'] = popularity
    
    # Extract the genres for each artist in the table
    for index, row in table.iterrows():
        for i in range(1, 6):
            artist_id = row[f'artist_id_{i}']
            if artist_id and artist_id != "none":
                artist = sp.artist(artist_id)
                genres = artist['genres']
                table.at[index, f'genre_{i}'] = ", ".join(genres)
            else:
                table.at[index, f'genre_{i}'] = "none"
    
    # Create the correct order of the columns
    current_cols = table.columns.tolist()
    new_cols = ['song', 'artist_1', 'artist_id_1', 'genre_1', 'album', 'label', 'popularity', 'total_tracks', 'album_id',
            'duration','date', 'date_album', 'imagen', 'url_song', 'artist_2', 'genre_2','artist_id_2',
            'artist_3','genre_3','artist_id_3', 'artist_4', 'artist_id_4',  'genre_4', 'artist_5', 'artist_id_5',
             'genre_5']
    table = table.reindex(columns=new_cols)
    
    return table

In [10]:
link = "https://open.spotify.com/playlist/2ZZQAzWW9nHlLh8QGWan3l?si=65eaa0f24dab4725"
playlist_80 = get_playlist_info(link)

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
playlist_80.sample(1)

,song,artist_1,artist_id_1,genre_1,album,label,popularity,total_tracks,album_id,duration,date,date_album,imagen,url_song,artist_2,genre_2,artist_id_2,artist_3,genre_3,artist_id_3,artist_4,artist_id_4,genre_4,artist_5,artist_id_5,genre_5
84,Eyes of the World - 2016 Remaster,Fleetwood Mac,08GQAI4eElDnROBrJRGE0X,"album rock, classic rock, rock, soft rock, yac...",Mirage (2016 Remaster),Rhino/Warner Records,47.0,12,2962eYkQa0R7EnIGZqeARS,226314,2022-10-01T07:01:14Z,1982-06-18,https://i.scdn.co/image/ab67616d0000b273b1751a...,https://open.spotify.com/track/0Fb7NBbIRP7Z6SV...,none,none,none,none,none,none,none,none,none,none,none,none


In [13]:
playlist_80.to_csv('Spotify_Playlist/1980_Spotify.csv', index=False)

In [14]:
link = "https://open.spotify.com/playlist/2Mrd1QzczV2yEwieR5v9Kp?si=64e6596090e6428c"
playlist_60_70 = get_playlist_info(link)

In [15]:
playlist_60_70.sample(1)

,song,artist_1,artist_id_1,genre_1,album,label,popularity,total_tracks,album_id,duration,date,date_album,imagen,url_song,artist_2,genre_2,artist_id_2,artist_3,genre_3,artist_id_3,artist_4,artist_id_4,genre_4,artist_5,artist_id_5,genre_5
18,Baby Boy,Status Quo,4gIdjgLlvgEOz7MexDZzpM,"blues rock, british blues, glam rock",Rockin' All Over The World,UMC (Universal Music Catalogue),57.0,28,5MsNoPWsiIbO6uz9KA6qDS,190986,2023-02-05T06:42:25Z,1977-11-11,https://i.scdn.co/image/ab67616d0000b273f3cbc5...,https://open.spotify.com/track/1LcdUwAgcWyS1bK...,none,none,none,none,none,none,none,none,none,none,none,none


In [17]:
playlist_60_70.to_csv('Spotify_Playlist/1960_1970_Spotify.csv', index=False)

In [18]:
link = "https://open.spotify.com/playlist/2Lq1zuWTgPBmaCXLw1kMeL?si=2a86773e503e4ee6"
playlist_90 = get_playlist_info(link)

In [19]:
playlist_90.sample(1)

,song,artist_1,artist_id_1,genre_1,album,label,popularity,total_tracks,album_id,duration,date,date_album,imagen,url_song,artist_2,genre_2,artist_id_2,artist_3,genre_3,artist_id_3,artist_4,artist_id_4,genre_4,artist_5,artist_id_5,genre_5
38,Sunjammer,Mike Oldfield,562Od3CffWedyz2BbeYWVn,"progressive rock, symphonic rock",Tubular Bells II,WM UK,45.0,14,0Z9Uio2xZCDztGCuozTPaH,152866,2023-03-09T08:31:39Z,1992,https://i.scdn.co/image/ab67616d0000b27397c14c...,https://open.spotify.com/track/4dTLQa0cMT6k88S...,none,none,none,none,none,none,none,none,none,none,none,none


In [20]:
playlist_90.to_csv('Spotify_Playlist/1990_Spotify.csv', index=False)

In [21]:
link = "https://open.spotify.com/playlist/0Jbc0Uh46Dvr03SSt0j2bs?si=5cf2a6352dd34889"
playlist_More_LPs = get_playlist_info(link)

In [22]:
playlist_More_LPs.sample(1)

,song,artist_1,artist_id_1,genre_1,album,label,popularity,total_tracks,album_id,duration,date,date_album,imagen,url_song,artist_2,genre_2,artist_id_2,artist_3,genre_3,artist_id_3,artist_4,artist_id_4,genre_4,artist_5,artist_id_5,genre_5
114,Un tempo piccolo,Mina,3HL1CyOnDLFJo1Rr8YBlKy,"classic italian pop, italian adult pop, ye ye",Cassiopea (Italian Songbook),PDU,32.0,15,1HpGfjRwXgfvSe7Dpdyl2l,247409,2023-04-18T17:02:25Z,2020-11-27,https://i.scdn.co/image/ab67616d0000b273d7bcef...,https://open.spotify.com/track/32XcGg71J8n8Bp4...,none,none,none,none,none,none,none,none,none,none,none,none


In [25]:
playlist_More_LPs.to_csv("Spotify_Playlist/MoreLPs_Spotify.csv", index=False)